### EEG Research Code 
> create by MinGeon Park

#### Set Default Path

In [1]:
#default_path setting
import os
default_path = '/home/devmin/Desktop/source/jupyter/research-EEG'
print(default_path)
print(os.chdir(default_path))
del default_path

/home/devmin/Desktop/source/jupyter/research-EEG
None


#### import mat data

In [2]:

#loadmat & import 문
from scipy import io
import matplotlib as mpl
import matplotlib.pylab as plt
import numpy as np

#data load
temp = io.loadmat('./data/sb1_eeg')

#### Type Check & EEG Data Get

> temp의 타입을 체크하고 dict타입이니 key를 확인

##### Type Check

In [3]:

#temp의 타입 확인
print(type(temp))
#dict 타입이니 key값 출력
print(temp.keys())

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'eeg'])


> __header__, __version__, __global__은 python에서 불러오는 도중 생기는 값으로 필요없음.
> 목적인 Data를 eeg데이터만을 dat변수에 저장

##### 

##### EEG data Get

> data변수는 numpy.ndarray로 확인되고, shape 필드를 통해 형태, data값을 확인

In [4]:
data = temp['eeg']
#eeg 데이터의 형태확인
print(type(data))
print(data.shape)
print(data[0][0])
#필요없어진 데이터 temp 삭제
del temp 

<class 'numpy.ndarray'>
(1, 1)
(array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[-586588.25 , -587784.75 , -587647.5  , ..., -288545.25 ,
        -288388.75 , -289359.5  ],
       [8293077.   , 8293056.   , 8293052.   , ..., 8292273.5  ,
        8292277.   , 8292280.   ],
       [ -79269.75 ,  -80505.   ,  -80313.75 , ...,  -18857.996,
         -18767.996,  -19776.746],
       ...,
       [-494490.5  , -495683.5  , -495491.5  , ..., -254509.75 ,
        -254321.5  , -255517.75 ],
       [-517539.5  , -518903.   , -518621.75 , ..., -273816.25 ,
        -273915.75 , -275170.75 ],
       [-539177.25 , -540427.25 , -540305.25 , ..., -272165.5  ,
        -271887.75 , -273722.5  ]], dtype=float32), array(['Driving simulation - bad condition'], dtype='<U34'), array([[512]], dtype=uint16))


#### Data extract

##### event & data로 분리
![matlab_data_view](../img/matlab_data_view.png)
> data 변수는 위의 매틀랩에 해당하는 부분이 3차원에 있다.

> event가 1인경우에 측정을 시작했다는 의미로, 1있는지 확인해야한다.

In [5]:
eeg_event = data[0][0][0][0]
eeg_data = data[0][0][1]
#eeg_event 출력
print(eeg_event)
#eeg_event에서 1이 있는지 확인
print(np.where(eeg_event==1))
#eeg_data가 정확한지 확인
print(eeg_data.shape)

[0 0 0 ... 0 0 0]
(array([1536]),)
(68, 923136)


##### RawEEGData로 분리

In [6]:
#eeg.event의 값을 데이터 시작값으로 저장
tp_startPt = np.where(eeg_event == 1)[0][0]
#eeg.data
RawEEGData = eeg_data[:,tp_startPt:]
#eeg.srate
SR = data[0][0][3][0][0]
#값 체크
print(RawEEGData.shape)
print(SR)

(68, 921600)
512


#### Data Reconstruction

> #RawEEGData를 세면 921600개인데, 이걸 SR로 나누고 그걸 다시 60으로 나누면 총 30분으로 떨어진다.

In [7]:
print(len(RawEEGData[0]))
MeasurementTime = int(len(RawEEGData[0])/SR/60)
print(MeasurementTime)
print(RawEEGData.shape)

921600
30
(68, 921600)


> ? 질문거리 :   
>  860160 : 890880 -> start:End  
>  29 -> temp_trial  
> 로 출력되는데, 그럼 총 0부터 30까지 총 30번 돌았을 것이다. 그런데 왜 tp_endPT는 최종값인 921600이 되지 않는가?  

In [8]:
#ProcessedData를 zeros를 통해 0으로 채워진 배열로 딱 맞게 생성
ProcessedData = np.zeros([30,64,30720])
print("\t",ProcessedData.shape,"-> ProcessedData shape")
for temp_trialNum in range(0,MeasurementTime):

    tp_startPt = temp_trialNum*(SR*60)
    tp_endPt = (SR*60) + temp_trialNum*(SR*60)

    #--------------------------진행사항 체크를 위한 출력------------------------------#
    print("|"+"-"*60+"|")
    print("|"+"\t",tp_startPt,":",tp_endPt,"-> start:End")
    print("|"+"\t",temp_trialNum,"-> temp_trial")
    print("|"+"-"*60+"|")
    #----------------------------------------------------------------------------#
    
        
    ProcessedData[temp_trialNum, :, :] = RawEEGData[:64, tp_startPt:tp_endPt]
    print(ProcessedData[temp_trialNum])

	 (30, 64, 30720) -> ProcessedData shape
|------------------------------------------------------------|


NameError: name 'tp_endPt' is not defined

#### matlab으로 생성한 reconstruction.mat 파일과 비교하기

In [ ]:
temp = io.loadmat('./data/reconstruction.mat')
print(temp["ProcessedData"].shape)
reconstruction_data = temp["ProcessedData"]
print(ProcessedData.shape)

#두 변수가 같은지 비교하는 matrix 비교 연산
print(np.array_equal(ProcessedData,reconstruction_data))

#최최최종값이 같은가?
print(ProcessedData[-1][-1][-1])
print(reconstruction_data[-1][-1][-1])

(30, 64, 30720)
(30, 64, 30720)
True
117888.5
117888.5
